# 搭建环境

In [1]:
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
import collections
import itertools
from typing import List, Tuple
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
import tensorflow as tf

PROJECT_ROOT_DIR = "."
H5_PATH = os.path.join(PROJECT_ROOT_DIR, "model_h5", "Gomoku-v0_tf_21")
os.makedirs(H5_PATH, exist_ok=True)

SavedModel_PATH = os.path.join(PROJECT_ROOT_DIR, "saved_model", "Gomoku-v0_tf_21")
os.makedirs(SavedModel_PATH, exist_ok=True)


EMPTY = 0
BLACK = 1
WHITE = -1
render_characters='+ox'
end='\n'

def strfboard(board:np.ndarray) -> str:
    
    s = ''
    for x in range(board.shape[0]):
        for y in range(board.shape[1]):
            c = render_characters[board[x][y]]
            s += c
        s += end
    return s[:-len(end)]

def is_index(board:np.ndarray, location:np.ndarray) -> bool:
    
    x, y = location
    return x in range(board.shape[0]) and y in range(board.shape[1])

@tf.function
def extend_board(board:tf.Tensor) -> tf.Tensor:
    
    boards = tf.stack([board,
            tf.experimental.numpy.rot90(board), 
            tf.experimental.numpy.rot90(board, k=2),
            tf.experimental.numpy.rot90(board, k=3), 
            tf.transpose(board), 
            tf.experimental.numpy.rot90(tf.transpose(board)),
            tf.experimental.numpy.rot90(tf.transpose(board),k=2), 
            tf.experimental.numpy.rot90(tf.transpose(board),k=3)])
    return boards

class KInARowEnv:

    def __init__(self, board_shape:int=3, target_length:int=3):
        
        self.target_length = target_length
        self.PASS = np.array([-1, 0],dtype=np.int32)
        if isinstance(board_shape, int):
            board_shape = (board_shape, board_shape)
        self.board = np.zeros(board_shape, dtype=np.int32)

    def reset(self) -> Tuple[np.ndarray, int]:
        
        self.board = np.zeros_like(self.board, dtype=np.int32)
        self.player = BLACK
        return self.board, self.player

    def is_valid(self, state:Tuple[np.ndarray, int], action:np.ndarray) -> bool:
        
        board, _ = state
        if not is_index(board, action):
            return False
        x, y = action
        return board[x, y] == EMPTY

    def get_valid(self, state:Tuple[np.ndarray, int]) -> np.ndarray:
        
        board, _ = state
        valid = np.zeros_like(board, dtype=np.int8)
        for x in range(board.shape[0]):
            for y in range(board.shape[1]):
                valid[x, y] = self.is_valid(state, np.array([x, y]))
        return valid

    def has_valid(self, state:Tuple[np.ndarray, int]) -> bool:
        
        board = state[0]
        for x in range(board.shape[0]):
            for y in range(board.shape[1]):
                if self.is_valid(state, np.array([x, y])):
                    return True
        return False

    def get_winner(self, state:Tuple[np.ndarray, int]) -> int:
        
        board, _ = state
        for player in [BLACK, WHITE]:
            for x in range(board.shape[0]):
                for y in range(board.shape[1]):
                    for dx, dy in [(1, -1), (1, 0), (1, 1), (0, 1)]: # loop on the 8 directions
                        xx, yy = x, y
                        for count in itertools.count():
                            if not is_index(board, (xx, yy)) or board[xx, yy] != player:
                                break
                            xx, yy = xx + dx, yy + dy
                        if count >= self.target_length:
                            return player
        for player in [BLACK, WHITE]:
            if self.has_valid((board, player)):
                return None
        return 0

    def get_next_state(self, state:Tuple[np.ndarray, int], action:np.ndarray) -> Tuple[np.ndarray, int]:
        
        board, player = state
        x, y = action
        if self.is_valid(state, action):
            board = np.array(board,dtype=np.int32)
            board[x, y] = player
        return board, -player

    def next_step(self, state:Tuple[np.ndarray, int],
                  action:np.ndarray) -> Tuple[np.ndarray, int, int, int]:
        
        if not self.is_valid(state, action):
            action = self.PASS
        while True:
            state = self.get_next_state(state, action)
            winner = self.get_winner(state)
            if winner is not None:
                return state, winner, 1
            if self.has_valid(state):
                break
            action = self.PASS
        return state, 0, 0

    def step(self, action:np.ndarray) -> Tuple[np.ndarray, int, int, int]:
        
        state = (self.board, self.player)
        next_state, winner, done = self.next_step(state, action)
        self.board, self.player = next_state
        return self.board, self.player, winner, done

# 五子棋 Gomoku-v0

## 环境实例

In [2]:
#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

## 智能体

In [2]:
def residual(x:tf.Tensor, filters:list, kernel_sizes:int=3, strides:int=1, activations:str='relu',
            regularizer:tf.keras.regularizers.Regularizer=tf.keras.regularizers.l2(1e-4)) -> tf.Tensor:
    
    shortcut = x
    for i, filte in enumerate(filters):
        kernel_size = kernel_sizes if isinstance(kernel_sizes, int) \
            else kernel_sizes[i]
        stride = strides if isinstance(strides, int) else strides[i]
        activation = activations if isinstance(activations, str) \
            else activations[i]
        z = tf.keras.layers.Conv2D(filte, kernel_size, strides=stride,
                               padding='same', kernel_regularizer=regularizer,
                               bias_regularizer=regularizer)(x)
        y = tf.keras.layers.BatchNormalization()(z)
        if i == len(filters) - 1:
            y = tf.keras.layers.Add()([shortcut, y])
        x = tf.keras.layers.Activation(activation)(y)
    return x

In [3]:
class AlphaZeroAgent:
    def __init__(self, env:object, epoches:int=1,
                 kwargs:dict={}, load:str=None, sim_count:int=800,
                 c_init:float=1.25, c_base:int=19652,
                 prior_exploration_fraction:float=0.25):
        
        self.env = env
        self.board = np.zeros_like(env.board,dtype=np.int32)
        self.epoches = epoches
        
        # Use the central storage strategy instead:
        distribution = tf.distribute.experimental.CentralStorageStrategy()
        
        with distribution.scope():
            if load is not None:
                self.net = tf.keras.models.load_model(load,
                    custom_objects={"categorical_crossentropy_2d": categorical_crossentropy_2d})
            else:
                self.net = self.build_network(**kwargs)
        
        self.reset_mcts()
        self.sim_count = sim_count # MCTS 次数
        self.c_init = c_init # PUCT 系数
        self.c_base = c_base # PUCT 系数
        self.prior_exploration_fraction = prior_exploration_fraction
        
    def build_network(self, conv_filters:list, residual_filters:List[list],
                      policy_filters:list,learning_rate:float=0.001,
                      regularizer:tf.keras.regularizers.Regularizer
                      =tf.keras.regularizers.l2(1e-4)) -> tf.keras.Model:
        
        # 公共部分
        inputs = tf.keras.Input(shape=self.board.shape)
        x = tf.keras.layers.Reshape(self.board.shape + (1,))(inputs)
        for conv_filter in conv_filters:
            z = tf.keras.layers.Conv2D(conv_filter, 3, padding='same',
                                   kernel_regularizer=regularizer,
                                   bias_regularizer=regularizer)(x)
            y = tf.keras.layers.BatchNormalization()(z)
            x = tf.keras.layers.ReLU()(y)
        for residual_filter in residual_filters:
            x = residual(x, filters=residual_filter, regularizer=regularizer)
        intermediates = x
        
        # 概率部分
        for policy_filter in policy_filters:
            z = tf.keras.layers.Conv2D(policy_filter, 3, padding='same',
                                   kernel_regularizer=regularizer,
                                   bias_regularizer=regularizer)(x)
            y = tf.keras.layers.BatchNormalization()(z)
            x = tf.keras.layers.ReLU()(y)
        logits = tf.keras.layers.Conv2D(1, 3, padding='same',
                kernel_regularizer=regularizer, bias_regularizer=regularizer)(x)
        flattens = tf.keras.layers.Flatten()(logits)
        softmaxs = tf.keras.layers.Softmax()(flattens)
        probs = tf.keras.layers.Reshape(self.board.shape)(softmaxs)
        
        # 价值部分
        z = tf.keras.layers.Conv2D(1, 3, padding='same',
                               kernel_regularizer=regularizer,
                               bias_regularizer=regularizer)(intermediates)
        y = tf.keras.layers.BatchNormalization()(z)
        x = tf.keras.layers.ReLU()(y)
        flattens = tf.keras.layers.Flatten()(x)
        vs = tf.keras.layers.Dense(1, activation=tf.keras.activations.tanh,
                               kernel_regularizer=regularizer,
                               bias_regularizer=regularizer)(flattens)
        
        model = tf.keras.Model(inputs=inputs, outputs=[probs, vs])
        
        @tf.function
        def categorical_crossentropy_2d(y_true:tf.Tensor,
                                        y_pred:tf.Tensor) -> tf.keras.losses.Loss:
            labels = tf.reshape(y_true, [-1, self.board.size])
            preds = tf.reshape(y_pred, [-1,self.board.size])
            return tf.keras.losses.categorical_crossentropy(labels, preds)
        
        loss = [categorical_crossentropy_2d, tf.keras.losses.MSE]
        optimizer = tf.keras.optimizers.Adam(learning_rate)
        model.compile(loss=loss,optimizer=optimizer)
        return model
    
    def reset_mcts(self):
        
        def zero_board_factory() -> np.ndarray: # 用于构造 default_dict
            return np.zeros_like(self.board, dtype=float)
        
        self.q = collections.defaultdict(zero_board_factory)
        # q 值估计: board -> board
        self.count = collections.defaultdict(zero_board_factory)
        # q 值计数: board -> board
        self.policy = {} # 策略：board -> board
        self.valid = {} # 有效位置：board -> board
        self.winner = {} # 赢家：board -> None or int
        
    def decide(self, observation:Tuple[np.ndarray, int]) -> Tuple[np.ndarray, np.ndarray]:
        
        # 计算策略
        board, player = observation
        canonical_board = player * board
        s = strfboard(canonical_board)
        while self.count[s].sum() < self.sim_count: # 多次 MCTS 搜索
            self.search(canonical_board, prior_noise=True)
        prob = self.count[s] / self.count[s].sum()
        
        # 采样
        location_index = np.random.choice(prob.size, p=prob.reshape(-1))
        location = np.unravel_index(location_index, prob.shape)
        return location, prob
    
    def learn(self, train_set:tf.data.Dataset):
        
        self.net.fit(train_set, verbose=0, epochs=self.epoches) # 训练
        self.reset_mcts()
        
    def search(self, board:np.ndarray, prior_noise:bool=False) -> np.ndarray: # MCTS 搜索
        
        s = strfboard(board)        
        if s not in self.winner:
            self.winner[s] = self.env.get_winner((board, BLACK)) # 计算赢家
        if self.winner[s] is not None: # 赢家确定的情况
            return self.winner[s]
        
        if s not in self.policy: # 未计算过策略的叶子节点
            pis, vs = self.net.predict(board[np.newaxis])
            pi, v = pis[0], vs[0]
            valid = self.env.get_valid((board, BLACK))
            masked_pi = pi * valid
            total_masked_pi = np.sum(masked_pi)
            if total_masked_pi <= 0: # 所有的有效动作都没有概率， 偶尔可能发生
                masked_pi = valid # workaround
                total_masked_pi = np.sum(masked_pi)
            self.policy[s] = masked_pi / total_masked_pi
            self.valid[s] = valid
            return v
        
        # PUCT 上届计算
        count_sum = self.count[s].sum()
        coef = (self.c_init + np.log1p((1 + count_sum) / self.c_base)) * \
            math.sqrt(count_sum) / (1. + self.count[s])
        
        if prior_noise: # 先验噪声
            alpha = 1. / self.valid[s].sum()
            noise = np.random.gamma(alpha, 1., board.shape)
            noise *= self.valid[s]
            noise /= noise.sum()
            prior = (1. - self.prior_exploration_fraction) * \
                self.policy[s] + \
                self.prior_exploration_fraction * noise
        else:
            prior = self.policy[s]
        
        ub = np.where(self.valid[s], self.q[s] + coef * prior, np.nan)
        location_index = np.nanargmax(ub)
        location = np.unravel_index(location_index, board.shape)
        
        (next_board, next_player), _, _ = self.env.next_step(
            (board, BLACK), np.array(location))
        next_canonical_board = next_player * next_board
        next_v = self.search(next_canonical_board) # 递归搜索
        v = next_player * next_v
        
        self.count[s][location] += 1
        self.q[s][location] += (v - self.q[s][location]) / \
                self.count[s][location]
        return v        

## 训练

In [5]:
"""
AlphaZero 参数， 可用来求解比较大型的问题 （如五子棋）
"""
train_iterations = 700000 # 训练迭代次数
train_episodes_per_iteration = 5000 # 每次迭代自我对弈回合数
epoches = 10 # 每回合进行几次批学习
batch_size = 4096 # 批学习的批大小
sim_count = 800 # MCTS需要的计数
net_kwargs = {}
net_kwargs['conv_filters'] = [256,]
net_kwargs['residual_filters'] = [[256, 256], ] * 19
net_kwargs['policy_filters'] = [256,]
shuffle_buffer_size = 10000

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 20
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
net_kwargs = {}
net_kwargs['conv_filters'] = [32,]
net_kwargs['residual_filters'] = [[32, 32],]
net_kwargs['policy_filters'] = [64,]
shuffle_buffer_size = 512

agent = AlphaZeroAgent(env=env, kwargs=net_kwargs, sim_count=sim_count,
                       epoches=epoches)

def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 0 回合 0: 收集到 184 条经验
训练 0 回合 1: 收集到 224 条经验
训练 0 回合 2: 收集到 216 条经验
训练 0 回合 3: 收集到 176 条经验
训练 0 回合 4: 收集到 192 条经验
训练 0 回合 5: 收集到 392 条经验
训练 0 回合 6: 收集到 200 条经验
训练 0 回合 7: 收集到 176 条经验
训练 0 回合 8: 收集到 184 条经验
训练 0 回合 9: 收集到 176 条经验
训练 0 回合 10: 收集到 168 条经验
训练 0 回合 11: 收集到 136 条经验
训练 0 回合 12: 收集到 192 条经验
训练 0 回合 13: 收集到 216 条经验
训练 0 回合 14: 收集到 168 条经验
训练 0 回合 15: 收集到 168 条经验
训练 0 回合 16: 收集到 392 条经验
训练 0 回合 17: 收集到 240 条经验
训练 0 回合 18: 收集到 272 条经验
训练 0 回合 19: 收集到 224 条经验
训练 0：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0000/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (1, 0)
+++++++
o++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (6, 4)
+++++++
o++++++
+++++++
+++++++
+++++++
+++++++

xxo+xx+
x+ooo+o
x+x++xx
ooo+oxo
+xo++o+
++++o++
+ox+xxo
第 29 步： 玩家 -1， 动作 (2, 1)
xxo+xx+
x+ooo+o
xxx++xx
ooo+oxo
+xo++o+
++++o++
+ox+xxo
第 30 步： 玩家 1， 动作 (4, 0)
xxo+xx+
x+ooo+o
xxx++xx
ooo+oxo
oxo++o+
++++o++
+ox+xxo
第 31 步： 玩家 -1， 动作 (1, 1)
xxo+xx+
xxooo+o
xxx++xx
ooo+oxo
oxo++o+
++++o++
+ox+xxo
第 32 步： 玩家 1， 动作 (3, 3)
xxo+xx+
xxooo+o
xxx++xx
oooooxo
oxo++o+
++++o++
+ox+xxo
赢家 1
训练 3 回合 0: 收集到 264 条经验
训练 3 回合 1: 收集到 296 条经验
训练 3 回合 2: 收集到 192 条经验
训练 3 回合 3: 收集到 176 条经验
训练 3 回合 4: 收集到 224 条经验
训练 3 回合 5: 收集到 232 条经验
训练 3 回合 6: 收集到 136 条经验
训练 3 回合 7: 收集到 144 条经验
训练 3 回合 8: 收集到 288 条经验
训练 3 回合 9: 收集到 184 条经验
训练 3 回合 10: 收集到 288 条经验
训练 3 回合 11: 收集到 200 条经验
训练 3 回合 12: 收集到 304 条经验
训练 3 回合 13: 收集到 192 条经验
训练 3 回合 14: 收集到 320 条经验
训练 3 回合 15: 收集到 344 条经验
训练 3 回合 16: 收集到 152 条经验
训练 3 回合 17: 收集到 232 条经验
训练 3 回合 18: 收集到 208 条经验
训练 3 回合 19: 收集到 208 条经验
训练 3：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0003/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 

+++++o+
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (0, 0)
x++++o+
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 2 步： 玩家 1， 动作 (3, 5)
x++++o+
+++++++
+++++++
+++++o+
+++++++
+++++++
+++++++
第 3 步： 玩家 -1， 动作 (2, 0)
x++++o+
+++++++
x++++++
+++++o+
+++++++
+++++++
+++++++
第 4 步： 玩家 1， 动作 (3, 3)
x++++o+
+++++++
x++++++
+++o+o+
+++++++
+++++++
+++++++
第 5 步： 玩家 -1， 动作 (2, 3)
x++++o+
+++++++
x++x+++
+++o+o+
+++++++
+++++++
+++++++
第 6 步： 玩家 1， 动作 (4, 5)
x++++o+
+++++++
x++x+++
+++o+o+
+++++o+
+++++++
+++++++
第 7 步： 玩家 -1， 动作 (1, 0)
x++++o+
x++++++
x++x+++
+++o+o+
+++++o+
+++++++
+++++++
第 8 步： 玩家 1， 动作 (2, 5)
x++++o+
x++++++
x++x+o+
+++o+o+
+++++o+
+++++++
+++++++
第 9 步： 玩家 -1， 动作 (3, 1)
x++++o+
x++++++
x++x+o+
+x+o+o+
+++++o+
+++++++
+++++++
第 10 步： 玩家 1， 动作 (4, 2)
x++++o+
x++++++
x++x+o+
+x+o+o+
++o++o+
+++++++
+++++++
第 11 步： 玩家 -1， 动作 (6, 3)
x++++o+
x++++++
x++x+o+
+x+o+o+
++o++o+
+++++++
+++x+++
第 12 步： 玩家 1， 动作 (0, 1)
xo+++o+
x++++++
x++x+o+
+x+o+o+
++o++o+
+++

训练 9 回合 14: 收集到 184 条经验
训练 9 回合 15: 收集到 168 条经验
训练 9 回合 16: 收集到 208 条经验
训练 9 回合 17: 收集到 184 条经验
训练 9 回合 18: 收集到 176 条经验
训练 9 回合 19: 收集到 208 条经验
训练 9：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0009/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 4)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
++++o++
第 1 步： 玩家 -1， 动作 (1, 4)
+++++++
++++x++
+++++++
+++++++
+++++++
+++++++
++++o++
第 2 步： 玩家 1， 动作 (3, 3)
+++++++
++++x++
+++++++
+++o+++
+++++++
+++++++
++++o++
第 3 步： 玩家 -1， 动作 (5, 0)
+++++++
++++x++
+++++++
+++o+++
+++++++
x++++++
++++o++
第 4 步： 玩家 1， 动作 (4, 2)
+++++++
++++x++
+++++++
+++o+++
++o++++
x++++++
++++o++
第 5 步： 玩家 -1， 动作 (2, 3)
+++++++
++++x++
+++x+++
+++o+++
++o++++
x++++++
++++o++
第 6 步： 玩家 1， 动作 (5, 5)
+++++++
++++x++
+++x+++
+++o+++
++o++++
x++++o+
++++o++
第 7 步： 玩家 -1， 动作 (1, 3)
+++++++
+++xx++
+++x+++
+++o+++
++o++++
x++++o+
++++o++
第 8 步： 玩家 1， 动作 (2, 4)
+++++++
+++xx++
+++xo++
+++o+++
++o++++
x++++o+
++++o+

训练 12 回合 0: 收集到 240 条经验
训练 12 回合 1: 收集到 224 条经验
训练 12 回合 2: 收集到 192 条经验
训练 12 回合 3: 收集到 288 条经验
训练 12 回合 4: 收集到 144 条经验
训练 12 回合 5: 收集到 112 条经验
训练 12 回合 6: 收集到 184 条经验
训练 12 回合 7: 收集到 144 条经验
训练 12 回合 8: 收集到 120 条经验
训练 12 回合 9: 收集到 312 条经验
训练 12 回合 10: 收集到 208 条经验
训练 12 回合 11: 收集到 104 条经验
训练 12 回合 12: 收集到 208 条经验
训练 12 回合 13: 收集到 208 条经验
训练 12 回合 14: 收集到 248 条经验
训练 12 回合 15: 收集到 216 条经验
训练 12 回合 16: 收集到 296 条经验
训练 12 回合 17: 收集到 208 条经验
训练 12 回合 18: 收集到 232 条经验
训练 12 回合 19: 收集到 296 条经验
训练 12：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0012/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (5, 0)
+++++++
+++++++
+++++++
+++++++
+++++++
o++++++
+++++++
第 1 步： 玩家 -1， 动作 (2, 0)
+++++++
+++++++
x++++++
+++++++
+++++++
o++++++
+++++++
第 2 步： 玩家 1， 动作 (1, 2)
+++++++
++o++++
x++++++
+++++++
+++++++
o++++++
+++++++
第 3 步： 玩家 -1， 动作 (2, 1)
+++++++
++o++++
xx+++++
+++++++
+++++++
o++++++
+++++++
第 4 步： 玩家 1， 动作 (1, 4)
+++++++
++o+o++
xx+++++
+

+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (0, 2)
++o++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (2, 6)
++o++++
+++++++
++++++x
+++++++
+++++++
+++++++
+++++++
第 2 步： 玩家 1， 动作 (5, 2)
++o++++
+++++++
++++++x
+++++++
+++++++
++o++++
+++++++
第 3 步： 玩家 -1， 动作 (3, 0)
++o++++
+++++++
++++++x
x++++++
+++++++
++o++++
+++++++
第 4 步： 玩家 1， 动作 (0, 4)
++o+o++
+++++++
++++++x
x++++++
+++++++
++o++++
+++++++
第 5 步： 玩家 -1， 动作 (3, 3)
++o+o++
+++++++
++++++x
x++x+++
+++++++
++o++++
+++++++
第 6 步： 玩家 1， 动作 (6, 3)
++o+o++
+++++++
++++++x
x++x+++
+++++++
++o++++
+++o+++
第 7 步： 玩家 -1， 动作 (3, 1)
++o+o++
+++++++
++++++x
xx+x+++
+++++++
++o++++
+++o+++
第 8 步： 玩家 1， 动作 (4, 1)
++o+o++
+++++++
++++++x
xx+x+++
+o+++++
++o++++
+++o+++
第 9 步： 玩家 -1， 动作 (5, 6)
++o+o++
+++++++
++++++x
xx+x+++
+o+++++
++o+++x
+++o+++
第 10 步： 玩家 1， 动作 (5, 0)
++o+o++
+++++++
++++++x
xx+x+++
+o+++++
o+o+++x
+++o+++
第 11 步： 玩家 -1， 动作 (4, 0)
++o+o++
+++++++
++++++x
xx+x+++
xo+++++
o+o+

+o+++++
x++++++
o+x++++
+++ox++
+++++++
+++++++
+++++++
第 6 步： 玩家 1， 动作 (0, 0)
oo+++++
x++++++
o+x++++
+++ox++
+++++++
+++++++
+++++++
第 7 步： 玩家 -1， 动作 (2, 5)
oo+++++
x++++++
o+x++x+
+++ox++
+++++++
+++++++
+++++++
第 8 步： 玩家 1， 动作 (2, 4)
oo+++++
x++++++
o+x+ox+
+++ox++
+++++++
+++++++
+++++++
第 9 步： 玩家 -1， 动作 (0, 5)
oo+++x+
x++++++
o+x+ox+
+++ox++
+++++++
+++++++
+++++++
第 10 步： 玩家 1， 动作 (4, 4)
oo+++x+
x++++++
o+x+ox+
+++ox++
++++o++
+++++++
+++++++
第 11 步： 玩家 -1， 动作 (5, 3)
oo+++x+
x++++++
o+x+ox+
+++ox++
++++o++
+++x+++
+++++++
第 12 步： 玩家 1， 动作 (2, 3)
oo+++x+
x++++++
o+xoox+
+++ox++
++++o++
+++x+++
+++++++
第 13 步： 玩家 -1， 动作 (4, 0)
oo+++x+
x++++++
o+xoox+
+++ox++
x+++o++
+++x+++
+++++++
第 14 步： 玩家 1， 动作 (6, 2)
oo+++x+
x++++++
o+xoox+
+++ox++
x+++o++
+++x+++
++o++++
第 15 步： 玩家 -1， 动作 (0, 4)
oo++xx+
x++++++
o+xoox+
+++ox++
x+++o++
+++x+++
++o++++
第 16 步： 玩家 1， 动作 (6, 5)
oo++xx+
x++++++
o+xoox+
+++ox++
x+++o++
+++x+++
++o++o+
第 17 步： 玩家 -1， 动作 (0, 2)
oox+xx+
x++++++
o+xoox+
+++ox++
x+++o+

## 评估

In [6]:
self_play_eval(env, agent)

+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (3, 1)
+++++++
+++++++
+++++++
+o+++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (6, 5)
+++++++
+++++++
+++++++
+o+++++
+++++++
+++++++
+++++x+
第 2 步： 玩家 1， 动作 (3, 4)
+++++++
+++++++
+++++++
+o++o++
+++++++
+++++++
+++++x+
第 3 步： 玩家 -1， 动作 (1, 4)
+++++++
++++x++
+++++++
+o++o++
+++++++
+++++++
+++++x+
第 4 步： 玩家 1， 动作 (3, 6)
+++++++
++++x++
+++++++
+o++o+o
+++++++
+++++++
+++++x+
第 5 步： 玩家 -1， 动作 (0, 5)
+++++x+
++++x++
+++++++
+o++o+o
+++++++
+++++++
+++++x+
第 6 步： 玩家 1， 动作 (5, 3)
+++++x+
++++x++
+++++++
+o++o+o
+++++++
+++o+++
+++++x+
第 7 步： 玩家 -1， 动作 (2, 3)
+++++x+
++++x++
+++x+++
+o++o+o
+++++++
+++o+++
+++++x+
第 8 步： 玩家 1， 动作 (6, 1)
+++++x+
++++x++
+++x+++
+o++o+o
+++++++
+++o+++
+o+++x+
第 9 步： 玩家 -1， 动作 (2, 2)
+++++x+
++++x++
++xx+++
+o++o+o
+++++++
+++o+++
+o+++x+
第 10 步： 玩家 1， 动作 (0, 4)
++++ox+
++++x++
++xx+++
+o++o+o
+++++++
+++o+++
+o+++x+
第 11 步： 玩家 -1， 动作 (2, 5)
++++ox+
++++x++
++xx+x+
+o++o+o
+++++++
+++o

# 应用

In [9]:
#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (2, 2)
+++++++
+++++++
++o++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (5, 3)
+++++++
+++++++
++o++++
+++++++
+++++++
+++x+++
+++++++
第 2 步： 玩家 1， 动作 (6, 1)
+++++++
+++++++
++o++++
+++++++
+++++++
+++x+++
+o+++++
第 3 步： 玩家 -1， 动作 (2, 4)
+++++++
+++++++
++o+x++
+++++++
+++++++
+++x+++
+o+++++
第 4 步： 玩家 1， 动作 (3, 0)
+++++++
+++++++
++o+x++
o++++++
+++++++
+++x+++
+o+++++
第 5 步： 玩家 -1， 动作 (1, 3)
+++++++
+++x+++
++o+x++
o++++++
+++++++
+++x+++
+o+++++
第 6 步： 玩家 1， 动作 (5, 2)
+++++++
+++x+++
++o+x++
o++++++
+++++++
++ox+++
+o+++++
第 7 步： 玩家 -1， 动作 (3, 6)
+++++++
+++x+++
++o+x++
o+++++x
+++++++
++ox+++
+o+++++
第 8 步： 玩家 1， 动作 (3, 1)
+++++++
+++x+++
++o+x++
oo++++x
+++++++
++ox+++


In [10]:
board = np.zeros_like(env.board, dtype=np.int32)
player = BLACK
observation = board, player

action, _ = agent.decide(observation)
action

(4, 1)

# 重复训练

## 训练1 train_iterations = 10 由于之前调试代码时第20次迭代已经训练完成，因此从第21次迭代训练

In [5]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 21
iteration_old_save_model = 20
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 21 回合 0: 收集到 232 条经验
训练 21 回合 1: 收集到 264 条经验
训练 21 回合 2: 收集到 280 条经验
训练 21 回合 3: 收集到 224 条经验
训练 21 回合 4: 收集到 280 条经验
训练 21 回合 5: 收集到 176 条经验
训练 21 回合 6: 收集到 248 条经验
训练 21 回合 7: 收集到 216 条经验
训练 21 回合 8: 收集到 144 条经验
训练 21 回合 9: 收集到 256 条经验
训练 21 回合 10: 收集到 184 条经验
训练 21 回合 11: 收集到 168 条经验
训练 21 回合 12: 收集到 264 条经验
训练 21 回合 13: 收集到 288 条经验
训练 21 回合 14: 收集到 200 条经验
训练 21 回合 15: 收集到 216 条经验
训练 21 回合 16: 收集到 192 条经验
训练 21 回合 17: 收集到 168 条经验
训练 21 回合 18: 收集到 264 条经验
训练 21 回合 19: 收集到 168 条经验
训练 21：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0021/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 3)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++o+++
第 1 步： 玩家 -1， 动作 (2, 2)
+++++++
+++++++
++x++++
++

训练 24 回合 5: 收集到 216 条经验
训练 24 回合 6: 收集到 152 条经验
训练 24 回合 7: 收集到 192 条经验
训练 24 回合 8: 收集到 272 条经验
训练 24 回合 9: 收集到 216 条经验
训练 24 回合 10: 收集到 224 条经验
训练 24 回合 11: 收集到 208 条经验
训练 24 回合 12: 收集到 168 条经验
训练 24 回合 13: 收集到 216 条经验
训练 24 回合 14: 收集到 192 条经验
训练 24 回合 15: 收集到 216 条经验
训练 24 回合 16: 收集到 296 条经验
训练 24 回合 17: 收集到 128 条经验
训练 24 回合 18: 收集到 264 条经验
训练 24 回合 19: 收集到 144 条经验
训练 24：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0024/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 5)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++o+
第 1 步： 玩家 -1， 动作 (0, 0)
x++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++o+
第 2 步： 玩家 1， 动作 (4, 4)
x++++++
+++++++
+++++++
+++++++
++++o++
+++++++
+++++o+
第 3 步： 玩家 -1， 动作 (2, 0)
x++++++
+++++++
x++++++
+++++++
++++o++
+++++++
+++++o+
第 4 步： 玩家 1， 动作 (4, 2)
x++++++
+++++++
x++++++
+++++++
++o+o++
+++++++
+++++o+
第 5 步： 玩家 -1， 动作 (0, 5)
x++++x+
+++++++
x++++++
+++++++
++o+o++
+++++++
+++++o+
第 6 步： 玩家

++++++o
++++x++
+o+o+oo
+ox++++
+xx+++x
xo+xx+o
+ox+o++
第 19 步： 玩家 -1， 动作 (1, 1)
++++++o
+x++x++
+o+o+oo
+ox++++
+xx+++x
xo+xx+o
+ox+o++
第 20 步： 玩家 1， 动作 (4, 3)
++++++o
+x++x++
+o+o+oo
+ox++++
+xxo++x
xo+xx+o
+ox+o++
第 21 步： 玩家 -1， 动作 (5, 2)
++++++o
+x++x++
+o+o+oo
+ox++++
+xxo++x
xoxxx+o
+ox+o++
第 22 步： 玩家 1， 动作 (1, 5)
++++++o
+x++xo+
+o+o+oo
+ox++++
+xxo++x
xoxxx+o
+ox+o++
第 23 步： 玩家 -1， 动作 (1, 2)
++++++o
+xx+xo+
+o+o+oo
+ox++++
+xxo++x
xoxxx+o
+ox+o++
第 24 步： 玩家 1， 动作 (2, 4)
++++++o
+xx+xo+
+o+oooo
+ox++++
+xxo++x
xoxxx+o
+ox+o++
第 25 步： 玩家 -1， 动作 (4, 4)
++++++o
+xx+xo+
+o+oooo
+ox++++
+xxox+x
xoxxx+o
+ox+o++
第 26 步： 玩家 1， 动作 (2, 2)
++++++o
+xx+xo+
+oooooo
+ox++++
+xxox+x
xoxxx+o
+ox+o++
赢家 1
训练 27 回合 0: 收集到 144 条经验
训练 27 回合 1: 收集到 296 条经验
训练 27 回合 2: 收集到 208 条经验
训练 27 回合 3: 收集到 176 条经验
训练 27 回合 4: 收集到 280 条经验
训练 27 回合 5: 收集到 200 条经验
训练 27 回合 6: 收集到 304 条经验
训练 27 回合 7: 收集到 152 条经验
训练 27 回合 8: 收集到 288 条经验
训练 27 回合 9: 收集到 152 条经验
训练 27 回合 10: 收集到 240 条经验
训练 27 回合 11: 收集到 240 条经验
训练 27

中间电脑死机了

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 0)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
o++++++
第 1 步： 玩家 -1， 动作 (2, 5)
+++++++
+++++++
+++++x+
+++++++
+++++++
+++++++
o++++++
第 2 步： 玩家 1， 动作 (1, 5)
+++++++
+++++o+
+++++x+
+++++++
+++++++
+++++++
o++++++
第 3 步： 玩家 -1， 动作 (6, 3)
+++++++
+++++o+
+++++x+
+++++++
+++++++
+++++++
o++x+++
第 4 步： 玩家 1， 动作 (3, 1)
+++++++
+++++o+
+++++x+
+o+++++
+++++++
+++++++
o++x+++
第 5 步： 玩家 -1， 动作 (3, 4)
+++++++
+++++o+
+++++x+
+o++x++
+++++++
+++++++
o++x+++
第 6 步： 玩家 1， 动作 (3, 5)
+++++++
+++++o+
+++++x+
+o++xo+
+++++++
+++++++
o++x+++
第 7 步： 玩家 -1， 动作 (4, 1)
+++++++
+++++o+
+++++x+
+o++xo+
+x+++++
+++++++
o++x+++
第 8 步： 玩家 1， 动作 (1, 2)
+++++++
++o++o+
+++++x+
+o++xo+
+x+++++
+++++++


## 训练2 train_iterations = 10 由于之前调试代码时第28次迭代已经训练完成，因此从第29次迭代训练

In [5]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 29
iteration_old_save_model = 28
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 29 回合 0: 收集到 176 条经验
训练 29 回合 1: 收集到 240 条经验
训练 29 回合 2: 收集到 248 条经验
训练 29 回合 3: 收集到 272 条经验
训练 29 回合 4: 收集到 296 条经验
训练 29 回合 5: 收集到 152 条经验
训练 29 回合 6: 收集到 184 条经验
训练 29 回合 7: 收集到 152 条经验
训练 29 回合 8: 收集到 192 条经验
训练 29 回合 9: 收集到 200 条经验
训练 29 回合 10: 收集到 136 条经验
训练 29 回合 11: 收集到 208 条经验
训练 29 回合 12: 收集到 240 条经验
训练 29 回合 13: 收集到 152 条经验
训练 29 回合 14: 收集到 200 条经验
训练 29 回合 15: 收集到 216 条经验
训练 29 回合 16: 收集到 224 条经验
训练 29 回合 17: 收集到 232 条经验
训练 29 回合 18: 收集到 192 条经验
训练 29 回合 19: 收集到 200 条经验
训练 29：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0029/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++o+++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (6, 1)
+++++++
+++++++
+++++++
++

训练 32 回合 9: 收集到 152 条经验
训练 32 回合 10: 收集到 296 条经验
训练 32 回合 11: 收集到 296 条经验
训练 32 回合 12: 收集到 280 条经验
训练 32 回合 13: 收集到 184 条经验
训练 32 回合 14: 收集到 184 条经验
训练 32 回合 15: 收集到 264 条经验
训练 32 回合 16: 收集到 152 条经验
训练 32 回合 17: 收集到 240 条经验
训练 32 回合 18: 收集到 200 条经验
训练 32 回合 19: 收集到 136 条经验
训练 32：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0032/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++o+++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (1, 2)
+++++++
++x++++
+++++++
+++o+++
+++++++
+++++++
+++++++
第 2 步： 玩家 1， 动作 (5, 1)
+++++++
++x++++
+++++++
+++o+++
+++++++
+o+++++
+++++++
第 3 步： 玩家 -1， 动作 (4, 0)
+++++++
++x++++
+++++++
+++o+++
x++++++
+o+++++
+++++++
第 4 步： 玩家 1， 动作 (1, 1)
+++++++
+ox++++
+++++++
+++o+++
x++++++
+o+++++
+++++++
第 5 步： 玩家 -1， 动作 (0, 0)
x++++++
+ox++++
+++++++
+++o+++
x++++++
+o+++++
+++++++
第 6 步： 玩家 1， 动作 (6, 1)
x++++++
+ox++++
+++++++
+++o+++
x++++++
+o+++++
+o+++++
第 7 步： 玩家 -1， 动作 (2, 5)
x+

+++o+o+
+++++++
o+x++x+
+x++ox+
+ox+xxo
o+xo+++
++++++o
第 17 步： 玩家 -1， 动作 (1, 6)
+++o+o+
++++++x
o+x++x+
+x++ox+
+ox+xxo
o+xo+++
++++++o
第 18 步： 玩家 1， 动作 (6, 5)
+++o+o+
++++++x
o+x++x+
+x++ox+
+ox+xxo
o+xo+++
+++++oo
第 19 步： 玩家 -1， 动作 (3, 2)
+++o+o+
++++++x
o+x++x+
+xx+ox+
+ox+xxo
o+xo+++
+++++oo
第 20 步： 玩家 1， 动作 (3, 3)
+++o+o+
++++++x
o+x++x+
+xxoox+
+ox+xxo
o+xo+++
+++++oo
第 21 步： 玩家 -1， 动作 (3, 0)
+++o+o+
++++++x
o+x++x+
xxxoox+
+ox+xxo
o+xo+++
+++++oo
第 22 步： 玩家 1， 动作 (6, 4)
+++o+o+
++++++x
o+x++x+
xxxoox+
+ox+xxo
o+xo+++
++++ooo
第 23 步： 玩家 -1， 动作 (2, 4)
+++o+o+
++++++x
o+x+xx+
xxxoox+
+ox+xxo
o+xo+++
++++ooo
第 24 步： 玩家 1， 动作 (5, 4)
+++o+o+
++++++x
o+x+xx+
xxxoox+
+ox+xxo
o+xoo++
++++ooo
第 25 步： 玩家 -1， 动作 (1, 4)
+++o+o+
++++x+x
o+x+xx+
xxxoox+
+ox+xxo
o+xoo++
++++ooo
第 26 步： 玩家 1， 动作 (5, 6)
+++o+o+
++++x+x
o+x+xx+
xxxoox+
+ox+xxo
o+xoo+o
++++ooo
第 27 步： 玩家 -1， 动作 (2, 3)
+++o+o+
++++x+x
o+xxxx+
xxxoox+
+ox+xxo
o+xoo+o
++++ooo
第 28 步： 玩家 1， 动作 (6, 2)
+++o+o+
++++x+x
o+xxxx+
xxxoox+
+o

++o+++x
++++++x
+++++++
+oo+oox
++o+x++
+o+x++o
x+++x++
第 15 步： 玩家 -1， 动作 (4, 6)
++o+++x
++++++x
+++++++
+oo+oox
++o+x+x
+o+x++o
x+++x++
第 16 步： 玩家 1， 动作 (3, 3)
++o+++x
++++++x
+++++++
+ooooox
++o+x+x
+o+x++o
x+++x++
赢家 1


In [7]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (5, 6)
+++++++
+++++++
+++++++
+++++++
+++++++
++++++o
+++++++
第 1 步： 玩家 -1， 动作 (4, 4)
+++++++
+++++++
+++++++
+++++++
++++x++
++++++o
+++++++
第 2 步： 玩家 1， 动作 (0, 1)
+o+++++
+++++++
+++++++
+++++++
++++x++
++++++o
+++++++
第 3 步： 玩家 -1， 动作 (1, 0)
+o+++++
x++++++
+++++++
+++++++
++++x++
++++++o
+++++++
第 4 步： 玩家 1， 动作 (5, 0)
+o+++++
x++++++
+++++++
+++++++
++++x++
o+++++o
+++++++
第 5 步： 玩家 -1， 动作 (3, 1)
+o+++++
x++++++
+++++++
+x+++++
++++x++
o+++++o
+++++++
第 6 步： 玩家 1， 动作 (3, 6)
+o+++++
x++++++
+++++++
+x++++o
++++x++
o+++++o
+++++++
第 7 步： 玩家 -1， 动作 (4, 1)
+o+++++
x++++++
+++++++
+x++++o
+x++x++
o+++++o
+++++++
第 8 步： 玩家 1， 动作 (2, 4)
+o+++++
x++++++
++++o++
+x++++o
+x++x++
o+++++o


## 训练3 train_iterations = 10 从第39次迭代训练

In [5]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 39
iteration_old_save_model = 38
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 39 回合 0: 收集到 224 条经验
训练 39 回合 1: 收集到 224 条经验
训练 39 回合 2: 收集到 232 条经验
训练 39 回合 3: 收集到 168 条经验
训练 39 回合 4: 收集到 200 条经验
训练 39 回合 5: 收集到 176 条经验
训练 39 回合 6: 收集到 280 条经验
训练 39 回合 7: 收集到 304 条经验
训练 39 回合 8: 收集到 128 条经验
训练 39 回合 9: 收集到 152 条经验
训练 39 回合 10: 收集到 152 条经验
训练 39 回合 11: 收集到 232 条经验
训练 39 回合 12: 收集到 168 条经验
训练 39 回合 13: 收集到 208 条经验
训练 39 回合 14: 收集到 128 条经验
训练 39 回合 15: 收集到 168 条经验
训练 39 回合 16: 收集到 392 条经验
训练 39 回合 17: 收集到 200 条经验
训练 39 回合 18: 收集到 288 条经验
训练 39 回合 19: 收集到 208 条经验
训练 39：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0039/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (1, 2)
+++++++
++o++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (3, 3)
+++++++
++o++++
+++++++
++

训练 42 回合 18: 收集到 168 条经验
训练 42 回合 19: 收集到 112 条经验
训练 42：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0042/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (5, 5)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++o+
+++++++
第 1 步： 玩家 -1， 动作 (5, 1)
+++++++
+++++++
+++++++
+++++++
+++++++
+x+++o+
+++++++
第 2 步： 玩家 1， 动作 (2, 0)
+++++++
+++++++
o++++++
+++++++
+++++++
+x+++o+
+++++++
第 3 步： 玩家 -1， 动作 (0, 6)
++++++x
+++++++
o++++++
+++++++
+++++++
+x+++o+
+++++++
第 4 步： 玩家 1， 动作 (1, 5)
++++++x
+++++o+
o++++++
+++++++
+++++++
+x+++o+
+++++++
第 5 步： 玩家 -1， 动作 (1, 1)
++++++x
+x+++o+
o++++++
+++++++
+++++++
+x+++o+
+++++++
第 6 步： 玩家 1， 动作 (2, 2)
++++++x
+x+++o+
o+o++++
+++++++
+++++++
+x+++o+
+++++++
第 7 步： 玩家 -1， 动作 (2, 5)
++++++x
+x+++o+
o+o++x+
+++++++
+++++++
+x+++o+
+++++++
第 8 步： 玩家 1， 动作 (5, 4)
++++++x
+x+++o+
o+o++x+
+++++++
+++++++
+x++oo+
+++++++
第 9 步： 玩家 -1， 动作 (0, 1)
+x++++x
+x+++o+
o+o++x+
+++++++
+++++++
+x++oo+
+++++++
第 10 步： 玩家 

+++++++
+++++++
+++++++
+++++++
+x+++o+
+++++++
+++++++
第 2 步： 玩家 1， 动作 (6, 3)
+++++++
+++++++
+++++++
+++++++
+x+++o+
+++++++
+++o+++
第 3 步： 玩家 -1， 动作 (4, 3)
+++++++
+++++++
+++++++
+++++++
+x+x+o+
+++++++
+++o+++
第 4 步： 玩家 1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++o+++
+x+x+o+
+++++++
+++o+++
第 5 步： 玩家 -1， 动作 (5, 4)
+++++++
+++++++
+++++++
+++o+++
+x+x+o+
++++x++
+++o+++
第 6 步： 玩家 1， 动作 (0, 1)
+o+++++
+++++++
+++++++
+++o+++
+x+x+o+
++++x++
+++o+++
第 7 步： 玩家 -1， 动作 (2, 6)
+o+++++
+++++++
++++++x
+++o+++
+x+x+o+
++++x++
+++o+++
第 8 步： 玩家 1， 动作 (3, 0)
+o+++++
+++++++
++++++x
o++o+++
+x+x+o+
++++x++
+++o+++
第 9 步： 玩家 -1， 动作 (3, 5)
+o+++++
+++++++
++++++x
o++o+x+
+x+x+o+
++++x++
+++o+++
第 10 步： 玩家 1， 动作 (1, 1)
+o+++++
+o+++++
++++++x
o++o+x+
+x+x+o+
++++x++
+++o+++
第 11 步： 玩家 -1， 动作 (1, 5)
+o+++++
+o+++x+
++++++x
o++o+x+
+x+x+o+
++++x++
+++o+++
第 12 步： 玩家 1， 动作 (3, 1)
+o+++++
+o+++x+
++++++x
oo+o+x+
+x+x+o+
++++x++
+++o+++
第 13 步： 玩家 -1， 动作 (3, 4)
+o+++++
+o+++x+
++++++x
oo+oxx+
+x+x+o+
++

训练 48 回合 12: 收集到 232 条经验
训练 48 回合 13: 收集到 344 条经验
训练 48 回合 14: 收集到 304 条经验
训练 48 回合 15: 收集到 192 条经验
训练 48 回合 16: 收集到 200 条经验
训练 48 回合 17: 收集到 280 条经验
训练 48 回合 18: 收集到 288 条经验
训练 48 回合 19: 收集到 232 条经验
训练 48：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0048/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++o+++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (4, 1)
+++++++
+++++++
+++++++
+++o+++
+x+++++
+++++++
+++++++
第 2 步： 玩家 1， 动作 (6, 6)
+++++++
+++++++
+++++++
+++o+++
+x+++++
+++++++
++++++o
第 3 步： 玩家 -1， 动作 (3, 6)
+++++++
+++++++
+++++++
+++o++x
+x+++++
+++++++
++++++o
第 4 步： 玩家 1， 动作 (2, 5)
+++++++
+++++++
+++++o+
+++o++x
+x+++++
+++++++
++++++o
第 5 步： 玩家 -1， 动作 (1, 1)
+++++++
+x+++++
+++++o+
+++o++x
+x+++++
+++++++
++++++o
第 6 步： 玩家 1， 动作 (5, 5)
+++++++
+x+++++
+++++o+
+++o++x
+x+++++
+++++o+
++++++o
第 7 步： 玩家 -1， 动作 (5, 0)
+++++++
+x+++++
+++++o+
+++o++x
+x+++++
x++++o+
++++++o
第 8 步： 玩家 1， 动作 (3, 

In [7]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 0)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
o++++++
第 1 步： 玩家 -1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++x+++
+++++++
+++++++
o++++++
第 2 步： 玩家 1， 动作 (1, 3)
+++++++
+++o+++
+++++++
+++x+++
+++++++
+++++++
o++++++
第 3 步： 玩家 -1， 动作 (4, 2)
+++++++
+++o+++
+++++++
+++x+++
++x++++
+++++++
o++++++
第 4 步： 玩家 1， 动作 (6, 1)
+++++++
+++o+++
+++++++
+++x+++
++x++++
+++++++
oo+++++
第 5 步： 玩家 -1， 动作 (0, 1)
+x+++++
+++o+++
+++++++
+++x+++
++x++++
+++++++
oo+++++
第 6 步： 玩家 1， 动作 (3, 1)
+x+++++
+++o+++
+++++++
+o+x+++
++x++++
+++++++
oo+++++
第 7 步： 玩家 -1， 动作 (6, 6)
+x+++++
+++o+++
+++++++
+o+x+++
++x++++
+++++++
oo++++x
第 8 步： 玩家 1， 动作 (3, 0)
+x+++++
+++o+++
+++++++
oo+x+++
++x++++
+++++++


## 训练4 train_iterations = 10 从第49次迭代训练

In [5]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 49
iteration_old_save_model = 48
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 49 回合 0: 收集到 160 条经验
训练 49 回合 1: 收集到 192 条经验
训练 49 回合 2: 收集到 176 条经验
训练 49 回合 3: 收集到 248 条经验
训练 49 回合 4: 收集到 240 条经验
训练 49 回合 5: 收集到 248 条经验
训练 49 回合 6: 收集到 248 条经验
训练 49 回合 7: 收集到 248 条经验
训练 49 回合 8: 收集到 176 条经验
训练 49 回合 9: 收集到 264 条经验
训练 49 回合 10: 收集到 168 条经验
训练 49 回合 11: 收集到 184 条经验
训练 49 回合 12: 收集到 160 条经验
训练 49 回合 13: 收集到 200 条经验
训练 49 回合 14: 收集到 336 条经验
训练 49 回合 15: 收集到 224 条经验
训练 49 回合 16: 收集到 224 条经验
训练 49 回合 17: 收集到 168 条经验
训练 49 回合 18: 收集到 200 条经验
训练 49 回合 19: 收集到 344 条经验
训练 49：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0049/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (1, 6)
+++++++
++++++o
+++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (1, 3)
+++++++
+++x++o
+++++++
++

o+++xx+
+++++o+
++++x+x
x++x+++
oo++oo+
+o+++++
+++++++
第 13 步： 玩家 -1， 动作 (5, 3)
o+++xx+
+++++o+
++++x+x
x++x+++
oo++oo+
+o+x+++
+++++++
第 14 步： 玩家 1， 动作 (6, 1)
o+++xx+
+++++o+
++++x+x
x++x+++
oo++oo+
+o+x+++
+o+++++
第 15 步： 玩家 -1， 动作 (4, 2)
o+++xx+
+++++o+
++++x+x
x++x+++
oox+oo+
+o+x+++
+o+++++
第 16 步： 玩家 1， 动作 (0, 3)
o++oxx+
+++++o+
++++x+x
x++x+++
oox+oo+
+o+x+++
+o+++++
第 17 步： 玩家 -1， 动作 (3, 5)
o++oxx+
+++++o+
++++x+x
x++x+x+
oox+oo+
+o+x+++
+o+++++
第 18 步： 玩家 1， 动作 (4, 6)
o++oxx+
+++++o+
++++x+x
x++x+x+
oox+ooo
+o+x+++
+o+++++
第 19 步： 玩家 -1， 动作 (6, 0)
o++oxx+
+++++o+
++++x+x
x++x+x+
oox+ooo
+o+x+++
xo+++++
第 20 步： 玩家 1， 动作 (3, 6)
o++oxx+
+++++o+
++++x+x
x++x+xo
oox+ooo
+o+x+++
xo+++++
第 21 步： 玩家 -1， 动作 (2, 5)
o++oxx+
+++++o+
++++xxx
x++x+xo
oox+ooo
+o+x+++
xo+++++
第 22 步： 玩家 1， 动作 (2, 1)
o++oxx+
+++++o+
+o++xxx
x++x+xo
oox+ooo
+o+x+++
xo+++++
第 23 步： 玩家 -1， 动作 (2, 3)
o++oxx+
+++++o+
+o+xxxx
x++x+xo
oox+ooo
+o+x+++
xo+++++
第 24 步： 玩家 1， 动作 (3, 1)
o++oxx+
+++++o+
+o+xxxx
xo+x+xo
oo

+++oo++
x+oox++
++x++++
++xox++
+++++x+
++++oo+
x+++x+o
第 16 步： 玩家 1， 动作 (2, 5)
+++oo++
x+oox++
++x++o+
++xox++
+++++x+
++++oo+
x+++x+o
第 17 步： 玩家 -1， 动作 (0, 5)
+++oox+
x+oox++
++x++o+
++xox++
+++++x+
++++oo+
x+++x+o
第 18 步： 玩家 1， 动作 (2, 1)
+++oox+
x+oox++
+ox++o+
++xox++
+++++x+
++++oo+
x+++x+o
第 19 步： 玩家 -1， 动作 (2, 4)
+++oox+
x+oox++
+ox+xo+
++xox++
+++++x+
++++oo+
x+++x+o
第 20 步： 玩家 1， 动作 (4, 3)
+++oox+
x+oox++
+ox+xo+
++xox++
+++o+x+
++++oo+
x+++x+o
第 21 步： 玩家 -1， 动作 (4, 4)
+++oox+
x+oox++
+ox+xo+
++xox++
+++oxx+
++++oo+
x+++x+o
第 22 步： 玩家 1， 动作 (5, 2)
+++oox+
x+oox++
+ox+xo+
++xox++
+++oxx+
++o+oo+
x+++x+o
第 23 步： 玩家 -1， 动作 (1, 1)
+++oox+
xxoox++
+ox+xo+
++xox++
+++oxx+
++o+oo+
x+++x+o
第 24 步： 玩家 1， 动作 (2, 3)
+++oox+
xxoox++
+oxoxo+
++xox++
+++oxx+
++o+oo+
x+++x+o
赢家 1
训练 55 回合 0: 收集到 208 条经验
训练 55 回合 1: 收集到 224 条经验
训练 55 回合 2: 收集到 256 条经验
训练 55 回合 3: 收集到 240 条经验
训练 55 回合 4: 收集到 256 条经验
训练 55 回合 5: 收集到 168 条经验
训练 55 回合 6: 收集到 216 条经验
训练 55 回合 7: 收集到 256 条经验
训练 55 回合 8: 收集到 224 条经验

++oo+++
+o+xx++
o++xoox
++xox++
o+x++++
+x++x+o
++x++o+
第 20 步： 玩家 1， 动作 (3, 6)
++oo+++
+o+xx++
o++xoox
++xox+o
o+x++++
+x++x+o
++x++o+
第 21 步： 玩家 -1， 动作 (5, 2)
++oo+++
+o+xx++
o++xoox
++xox+o
o+x++++
+xx+x+o
++x++o+
第 22 步： 玩家 1， 动作 (6, 1)
++oo+++
+o+xx++
o++xoox
++xox+o
o+x++++
+xx+x+o
+ox++o+
第 23 步： 玩家 -1， 动作 (1, 0)
++oo+++
xo+xx++
o++xoox
++xox+o
o+x++++
+xx+x+o
+ox++o+
第 24 步： 玩家 1， 动作 (5, 0)
++oo+++
xo+xx++
o++xoox
++xox+o
o+x++++
oxx+x+o
+ox++o+
第 25 步： 玩家 -1， 动作 (2, 2)
++oo+++
xo+xx++
o+xxoox
++xox+o
o+x++++
oxx+x+o
+ox++o+
赢家 -1
训练 58 回合 0: 收集到 232 条经验
训练 58 回合 1: 收集到 168 条经验
训练 58 回合 2: 收集到 264 条经验
训练 58 回合 3: 收集到 248 条经验
训练 58 回合 4: 收集到 160 条经验
训练 58 回合 5: 收集到 208 条经验
训练 58 回合 6: 收集到 176 条经验
训练 58 回合 7: 收集到 136 条经验
训练 58 回合 8: 收集到 216 条经验
训练 58 回合 9: 收集到 288 条经验
训练 58 回合 10: 收集到 216 条经验
训练 58 回合 11: 收集到 168 条经验
训练 58 回合 12: 收集到 248 条经验
训练 58 回合 13: 收集到 152 条经验
训练 58 回合 14: 收集到 200 条经验
训练 58 回合 15: 收集到 168 条经验
训练 58 回合 16: 收集到 176 条经验
训练 58 回合 17: 收集到 272 条经验
训练 58 回合 18: 收集

In [7]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (5, 6)
+++++++
+++++++
+++++++
+++++++
+++++++
++++++o
+++++++
第 1 步： 玩家 -1， 动作 (6, 4)
+++++++
+++++++
+++++++
+++++++
+++++++
++++++o
++++x++
第 2 步： 玩家 1， 动作 (3, 3)
+++++++
+++++++
+++++++
+++o+++
+++++++
++++++o
++++x++
第 3 步： 玩家 -1， 动作 (1, 3)
+++++++
+++x+++
+++++++
+++o+++
+++++++
++++++o
++++x++
第 4 步： 玩家 1， 动作 (4, 2)
+++++++
+++x+++
+++++++
+++o+++
++o++++
++++++o
++++x++
第 5 步： 玩家 -1， 动作 (5, 5)
+++++++
+++x+++
+++++++
+++o+++
++o++++
+++++xo
++++x++
第 6 步： 玩家 1， 动作 (4, 0)
+++++++
+++x+++
+++++++
+++o+++
o+o++++
+++++xo
++++x++
第 7 步： 玩家 -1， 动作 (2, 3)
+++++++
+++x+++
+++x+++
+++o+++
o+o++++
+++++xo
++++x++
第 8 步： 玩家 1， 动作 (4, 6)
+++++++
+++x+++
+++x+++
+++o+++
o+o+++o
+++++xo


## 训练5 train_iterations = 10 从第59次迭代训练

In [4]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [5]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 59
iteration_old_save_model = 58
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 59 回合 0: 收集到 216 条经验
训练 59 回合 1: 收集到 296 条经验
训练 59 回合 2: 收集到 200 条经验
训练 59 回合 3: 收集到 184 条经验
训练 59 回合 4: 收集到 208 条经验
训练 59 回合 5: 收集到 264 条经验
训练 59 回合 6: 收集到 248 条经验
训练 59 回合 7: 收集到 192 条经验
训练 59 回合 8: 收集到 120 条经验
训练 59 回合 9: 收集到 160 条经验
训练 59 回合 10: 收集到 168 条经验
训练 59 回合 11: 收集到 280 条经验
训练 59 回合 12: 收集到 240 条经验
训练 59 回合 13: 收集到 200 条经验
训练 59 回合 14: 收集到 168 条经验
训练 59 回合 15: 收集到 256 条经验
训练 59 回合 16: 收集到 184 条经验
训练 59 回合 17: 收集到 248 条经验
训练 59 回合 18: 收集到 280 条经验
训练 59 回合 19: 收集到 160 条经验
训练 59：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0059/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 4)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
++++o++
第 1 步： 玩家 -1， 动作 (2, 1)
+++++++
+++++++
+x+++++
++

xo++xo+
x+++x++
oxooxxo
oxo++++
++xxo+x
++ox++o
o++++++
第 24 步： 玩家 1， 动作 (5, 5)
xo++xo+
x+++x++
oxooxxo
oxo++++
++xxo+x
++ox+oo
o++++++
第 25 步： 玩家 -1， 动作 (4, 1)
xo++xo+
x+++x++
oxooxxo
oxo++++
+xxxo+x
++ox+oo
o++++++
第 26 步： 玩家 1， 动作 (1, 3)
xo++xo+
x++ox++
oxooxxo
oxo++++
+xxxo+x
++ox+oo
o++++++
第 27 步： 玩家 -1， 动作 (6, 4)
xo++xo+
x++ox++
oxooxxo
oxo++++
+xxxo+x
++ox+oo
o+++x++
第 28 步： 玩家 1， 动作 (6, 6)
xo++xo+
x++ox++
oxooxxo
oxo++++
+xxxo+x
++ox+oo
o+++x+o
第 29 步： 玩家 -1， 动作 (1, 1)
xo++xo+
xx+ox++
oxooxxo
oxo++++
+xxxo+x
++ox+oo
o+++x+o
第 30 步： 玩家 1， 动作 (5, 0)
xo++xo+
xx+ox++
oxooxxo
oxo++++
+xxxo+x
o+ox+oo
o+++x+o
第 31 步： 玩家 -1， 动作 (5, 1)
xo++xo+
xx+ox++
oxooxxo
oxo++++
+xxxo+x
oxox+oo
o+++x+o
赢家 -1
训练 62 回合 0: 收集到 312 条经验
训练 62 回合 1: 收集到 168 条经验
训练 62 回合 2: 收集到 224 条经验
训练 62 回合 3: 收集到 328 条经验
训练 62 回合 4: 收集到 224 条经验
训练 62 回合 5: 收集到 264 条经验
训练 62 回合 6: 收集到 296 条经验
训练 62 回合 7: 收集到 184 条经验
训练 62 回合 8: 收集到 144 条经验
训练 62 回合 9: 收集到 200 条经验
训练 62 回合 10: 收集到 336 条经验
训练 62 回合 11: 收集到 224 条经验
训练 6

++o+o++
+++++++
oo++x+x
x+o+x++
++++x++
xooo++x
++oxo+x
第 19 步： 玩家 -1， 动作 (3, 1)
++o+o++
+++++++
oo++x+x
xxo+x++
++++x++
xooo++x
++oxo+x
第 20 步： 玩家 1， 动作 (1, 6)
++o+o++
++++++o
oo++x+x
xxo+x++
++++x++
xooo++x
++oxo+x
第 21 步： 玩家 -1， 动作 (0, 0)
x+o+o++
++++++o
oo++x+x
xxo+x++
++++x++
xooo++x
++oxo+x
第 22 步： 玩家 1， 动作 (4, 5)
x+o+o++
++++++o
oo++x+x
xxo+x++
++++xo+
xooo++x
++oxo+x
第 23 步： 玩家 -1， 动作 (3, 5)
x+o+o++
++++++o
oo++x+x
xxo+xx+
++++xo+
xooo++x
++oxo+x
第 24 步： 玩家 1， 动作 (2, 3)
x+o+o++
++++++o
oo+ox+x
xxo+xx+
++++xo+
xooo++x
++oxo+x
第 25 步： 玩家 -1， 动作 (1, 1)
x+o+o++
+x++++o
oo+ox+x
xxo+xx+
++++xo+
xooo++x
++oxo+x
第 26 步： 玩家 1， 动作 (6, 1)
x+o+o++
+x++++o
oo+ox+x
xxo+xx+
++++xo+
xooo++x
+ooxo+x
第 27 步： 玩家 -1， 动作 (5, 5)
x+o+o++
+x++++o
oo+ox+x
xxo+xx+
++++xo+
xooo+xx
+ooxo+x
第 28 步： 玩家 1， 动作 (3, 3)
x+o+o++
+x++++o
oo+ox+x
xxooxx+
++++xo+
xooo+xx
+ooxo+x
第 29 步： 玩家 -1， 动作 (2, 5)
x+o+o++
+x++++o
oo+oxxx
xxooxx+
++++xo+
xooo+xx
+ooxo+x
第 30 步： 玩家 1， 动作 (2, 2)
x+o+o++
+x++++o
ooooxxx
xxooxx+
++

xxxox++
++++++x
oxxoo+o
+o+ox+o
x++o+xx
+o+oxoo
++++x+o
第 27 步： 玩家 -1， 动作 (1, 2)
xxxox++
++x+++x
oxxoo+o
+o+ox+o
x++o+xx
+o+oxoo
++++x+o
第 28 步： 玩家 1， 动作 (1, 1)
xxxox++
+ox+++x
oxxoo+o
+o+ox+o
x++o+xx
+o+oxoo
++++x+o
第 29 步： 玩家 -1， 动作 (6, 5)
xxxox++
+ox+++x
oxxoo+o
+o+ox+o
x++o+xx
+o+oxoo
++++xxo
第 30 步： 玩家 1， 动作 (1, 0)
xxxox++
oox+++x
oxxoo+o
+o+ox+o
x++o+xx
+o+oxoo
++++xxo
第 31 步： 玩家 -1， 动作 (5, 2)
xxxox++
oox+++x
oxxoo+o
+o+ox+o
x++o+xx
+oxoxoo
++++xxo
第 32 步： 玩家 1， 动作 (1, 4)
xxxox++
oox+o+x
oxxoo+o
+o+ox+o
x++o+xx
+oxoxoo
++++xxo
第 33 步： 玩家 -1， 动作 (2, 5)
xxxox++
oox+o+x
oxxooxo
+o+ox+o
x++o+xx
+oxoxoo
++++xxo
第 34 步： 玩家 1， 动作 (1, 3)
xxxox++
ooxoo+x
oxxooxo
+o+ox+o
x++o+xx
+oxoxoo
++++xxo
赢家 1
训练 68 回合 0: 收集到 208 条经验
训练 68 回合 1: 收集到 264 条经验
训练 68 回合 2: 收集到 312 条经验
训练 68 回合 3: 收集到 224 条经验
训练 68 回合 4: 收集到 120 条经验
训练 68 回合 5: 收集到 208 条经验
训练 68 回合 6: 收集到 216 条经验
训练 68 回合 7: 收集到 304 条经验
训练 68 回合 8: 收集到 160 条经验
训练 68 回合 9: 收集到 216 条经验
训练 68 回合 10: 收集到 128 条经验
训练 68 回合 11: 收集到 176 条经验
训练 68

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 0)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
o++++++
第 1 步： 玩家 -1， 动作 (3, 0)
+++++++
+++++++
+++++++
x++++++
+++++++
+++++++
o++++++
第 2 步： 玩家 1， 动作 (5, 4)
+++++++
+++++++
+++++++
x++++++
+++++++
++++o++
o++++++
第 3 步： 玩家 -1， 动作 (5, 0)
+++++++
+++++++
+++++++
x++++++
+++++++
x+++o++
o++++++
第 4 步： 玩家 1， 动作 (1, 0)
+++++++
o++++++
+++++++
x++++++
+++++++
x+++o++
o++++++
第 5 步： 玩家 -1， 动作 (5, 3)
+++++++
o++++++
+++++++
x++++++
+++++++
x++xo++
o++++++
第 6 步： 玩家 1， 动作 (1, 3)
+++++++
o++o+++
+++++++
x++++++
+++++++
x++xo++
o++++++
第 7 步： 玩家 -1， 动作 (4, 0)
+++++++
o++o+++
+++++++
x++++++
x++++++
x++xo++
o++++++
第 8 步： 玩家 1， 动作 (3, 3)
+++++++
o++o+++
+++++++
x++o+++
x++++++
x++xo++


## 训练6 train_iterations = 10 从第69次迭代训练¶

In [4]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [5]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 69
iteration_old_save_model = 68
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 69 回合 0: 收集到 152 条经验
训练 69 回合 1: 收集到 216 条经验
训练 69 回合 2: 收集到 184 条经验
训练 69 回合 3: 收集到 248 条经验
训练 69 回合 4: 收集到 240 条经验
训练 69 回合 5: 收集到 160 条经验
训练 69 回合 6: 收集到 304 条经验
训练 69 回合 7: 收集到 168 条经验
训练 69 回合 8: 收集到 184 条经验
训练 69 回合 9: 收集到 248 条经验
训练 69 回合 10: 收集到 184 条经验
训练 69 回合 11: 收集到 216 条经验
训练 69 回合 12: 收集到 160 条经验
训练 69 回合 13: 收集到 184 条经验
训练 69 回合 14: 收集到 392 条经验
训练 69 回合 15: 收集到 184 条经验
训练 69 回合 16: 收集到 272 条经验
训练 69 回合 17: 收集到 208 条经验
训练 69 回合 18: 收集到 216 条经验
训练 69 回合 19: 收集到 216 条经验
训练 69：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0069/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (2, 0)
+++++++
+++++++
o++++++
+++++++
+++++++
+++++++
+++++++
第 1 步： 玩家 -1， 动作 (4, 1)
+++++++
+++++++
o++++++
++

+++++++
+++++++
+++++++
+++o+++
+++++++
x++++++
+++o+++
第 3 步： 玩家 -1， 动作 (1, 3)
+++++++
+++x+++
+++++++
+++o+++
+++++++
x++++++
+++o+++
第 4 步： 玩家 1， 动作 (3, 5)
+++++++
+++x+++
+++++++
+++o+o+
+++++++
x++++++
+++o+++
第 5 步： 玩家 -1， 动作 (1, 0)
+++++++
x++x+++
+++++++
+++o+o+
+++++++
x++++++
+++o+++
第 6 步： 玩家 1， 动作 (4, 3)
+++++++
x++x+++
+++++++
+++o+o+
+++o+++
x++++++
+++o+++
第 7 步： 玩家 -1， 动作 (6, 6)
+++++++
x++x+++
+++++++
+++o+o+
+++o+++
x++++++
+++o++x
第 8 步： 玩家 1， 动作 (5, 3)
+++++++
x++x+++
+++++++
+++o+o+
+++o+++
x++o+++
+++o++x
第 9 步： 玩家 -1， 动作 (6, 2)
+++++++
x++x+++
+++++++
+++o+o+
+++o+++
x++o+++
++xo++x
第 10 步： 玩家 1， 动作 (4, 6)
+++++++
x++x+++
+++++++
+++o+o+
+++o++o
x++o+++
++xo++x
第 11 步： 玩家 -1， 动作 (3, 0)
+++++++
x++x+++
+++++++
x++o+o+
+++o++o
x++o+++
++xo++x
第 12 步： 玩家 1， 动作 (6, 5)
+++++++
x++x+++
+++++++
x++o+o+
+++o++o
x++o+++
++xo+ox
第 13 步： 玩家 -1， 动作 (1, 2)
+++++++
x+xx+++
+++++++
x++o+o+
+++o++o
x++o+++
++xo+ox
第 14 步： 玩家 1， 动作 (2, 3)
+++++++
x+xx+++
+++o+++
x++o+o+
+++o++o
x

+++++++
+++++++
+++++++
+++x+o+
+++++++
+++++++
++++o++
第 3 步： 玩家 -1， 动作 (6, 1)
+++++++
+++++++
+++++++
+++x+o+
+++++++
+++++++
+x++o++
第 4 步： 玩家 1， 动作 (6, 3)
+++++++
+++++++
+++++++
+++x+o+
+++++++
+++++++
+x+oo++
第 5 步： 玩家 -1， 动作 (2, 3)
+++++++
+++++++
+++x+++
+++x+o+
+++++++
+++++++
+x+oo++
第 6 步： 玩家 1， 动作 (4, 3)
+++++++
+++++++
+++x+++
+++x+o+
+++o+++
+++++++
+x+oo++
第 7 步： 玩家 -1， 动作 (5, 2)
+++++++
+++++++
+++x+++
+++x+o+
+++o+++
++x++++
+x+oo++
第 8 步： 玩家 1， 动作 (5, 3)
+++++++
+++++++
+++x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 9 步： 玩家 -1， 动作 (1, 3)
+++++++
+++x+++
+++x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 10 步： 玩家 1， 动作 (2, 1)
+++++++
+++x+++
+o+x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 11 步： 玩家 -1， 动作 (0, 4)
++++x++
+++x+++
+o+x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 12 步： 玩家 1， 动作 (1, 2)
++++x++
++ox+++
+o+x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 13 步： 玩家 -1， 动作 (2, 0)
++++x++
++ox+++
xo+x+++
+++x+o+
+++o+++
++xo+++
+x+oo++
第 14 步： 玩家 1， 动作 (4, 2)
++++x++
++ox+++
xo+x+++
+++x+o+
++oo+++
+

+++x+++
xo+o+x+
++xoxxo
++ox+ox
++xoxxx
++ooo++
oo++x+o
第 26 步： 玩家 1， 动作 (3, 1)
+++x+++
xo+o+x+
++xoxxo
+oox+ox
++xoxxx
++ooo++
oo++x+o
第 27 步： 玩家 -1， 动作 (0, 6)
+++x++x
xo+o+x+
++xoxxo
+oox+ox
++xoxxx
++ooo++
oo++x+o
赢家 -1
训练 78 回合 0: 收集到 120 条经验
训练 78 回合 1: 收集到 192 条经验
训练 78 回合 2: 收集到 304 条经验
训练 78 回合 3: 收集到 264 条经验
训练 78 回合 4: 收集到 128 条经验
训练 78 回合 5: 收集到 208 条经验
训练 78 回合 6: 收集到 184 条经验
训练 78 回合 7: 收集到 256 条经验
训练 78 回合 8: 收集到 168 条经验
训练 78 回合 9: 收集到 216 条经验
训练 78 回合 10: 收集到 336 条经验
训练 78 回合 11: 收集到 256 条经验
训练 78 回合 12: 收集到 176 条经验
训练 78 回合 13: 收集到 144 条经验
训练 78 回合 14: 收集到 192 条经验
训练 78 回合 15: 收集到 208 条经验
训练 78 回合 16: 收集到 176 条经验
训练 78 回合 17: 收集到 200 条经验
训练 78 回合 18: 收集到 256 条经验
训练 78 回合 19: 收集到 256 条经验
训练 78：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0078/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (5, 4)
+++++++
+++++++
+++++++
+++++++
+++++++
++++o++
+++++++
第 1 步： 玩家 -1， 动作 (0, 1)
+x+++++
+++++++
+++++++
+++++++
+++++++


In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 0)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
o++++++
第 1 步： 玩家 -1， 动作 (2, 5)
+++++++
+++++++
+++++x+
+++++++
+++++++
+++++++
o++++++
第 2 步： 玩家 1， 动作 (2, 4)
+++++++
+++++++
++++ox+
+++++++
+++++++
+++++++
o++++++
第 3 步： 玩家 -1， 动作 (1, 4)
+++++++
++++x++
++++ox+
+++++++
+++++++
+++++++
o++++++
第 4 步： 玩家 1， 动作 (1, 5)
+++++++
++++xo+
++++ox+
+++++++
+++++++
+++++++
o++++++
第 5 步： 玩家 -1， 动作 (1, 6)
+++++++
++++xox
++++ox+
+++++++
+++++++
+++++++
o++++++
第 6 步： 玩家 1， 动作 (6, 5)
+++++++
++++xox
++++ox+
+++++++
+++++++
+++++++
o++++o+
第 7 步： 玩家 -1， 动作 (1, 2)
+++++++
++x+xox
++++ox+
+++++++
+++++++
+++++++
o++++o+
第 8 步： 玩家 1， 动作 (6, 2)
+++++++
++x+xox
++++ox+
+++++++
+++++++
+++++++


## 训练7 train_iterations = 10 从第79次迭代训练

In [4]:
def env_reset(verbose:bool) -> Tuple[np.ndarray, np.ndarray]:
    
    board, player = env.reset()
    return (np.array(board, np.int32), np.array(player, np.int32))

def tf_env_reset(verbose:tf.bool) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_reset, inp=[verbose],
                            Tout=[tf.int32, tf.int32])

def env_step(action:np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    
    board, player, winner, done = env.step(action)
    return (board, np.array(player,np.int32),
            np.array(winner, np.int32),
            np.array(done, np.int32))

def tf_env_step(action:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(env_step, [action],
                            [tf.int32, tf.int32 , tf.int32, tf.int32])

def agent_decide(board:np.ndarray, player:int) -> Tuple[np.ndarray, np.ndarray]:
    
    observation = board, player
    action, prob = agent.decide(observation)
    return (np.array(action,dtype=np.int32),
            np.array(prob, dtype=np.float32))

def tf_agent_decide(board:tf.Tensor, player:tf.Tensor) -> List[tf.Tensor]:
    
    return tf.numpy_function(agent_decide, [board, player],
                            [tf.int32, tf.float32])

@tf.function
def self_play(env:object, agent:object) -> List[tf.Tensor]:
    
    players = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    boards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
    winners = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    board, player = tf_env_reset(False)
    
    # 15 * 15 = 225
    t = 0
    for step in tf.range(240):
        
        action, prob = tf_agent_decide(board, player)
        board,player,winner,done = tf_env_step(action)
        boards_stack = extend_board(board)
        
        for i in range(8):
            players = players.write(t, player)
            boards = boards.write(t, boards_stack[i])
            probs = probs.write(t, prob)
            t += 1
        
        if tf.cast(done, tf.bool): 
            winners=winners.write(0, winner)
            break
    
    winners = winners.stack()
    players = players.stack()
    boards = boards.stack()
    probs = probs.stack()
    return players, boards, probs, winners

def self_play_eval(env:object, agent:object):
    
    board, player = env.reset()
    
    # 15 * 15 = 225    
    for step in range(240):
        
        observation = board, player
        action, prob = agent.decide(observation)
        
        print(strfboard(board))
        print('第 {} 步： 玩家 {}， 动作 {}'.format(step,player,action))
        
        board,player,winner,done = env.step(action)
        
        if done:
            print(strfboard(board))
            print('赢家 {}'.format(winner)) 
            break

@tf.function
def preprocess(trajectory:dict) -> List[tf.Tensor]:
    
    player = trajectory["player"]
    board = trajectory["board"]
    prob = trajectory["prob"]
    winner = trajectory["winner"]
    
    cononical_boards = board * player
    vs = (player * winner)
    return cononical_boards, (prob, vs)

In [5]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

"""
小规模参数， 用来初步求解比较小的问题（如井字棋）
"""
train_iterations = 10
train_episodes_per_iteration = 20
epoches = 10
batch_size = 64
sim_count = 200
shuffle_buffer_size = 512
iteration_old = 79
iteration_old_save_model = 78
current_model_path = os.path.join(H5_PATH, f"model_{iteration_old_save_model:04}iterations.h5")

agent = AlphaZeroAgent(env=env, load=current_model_path, sim_count=sim_count,
                       epoches=epoches)

for iteration in range(train_iterations):
    
    # 自我对弈
    trajectorys = {}
    ls_players = []
    ls_boards = []
    ls_probs = []
    ls_winners = []
    
    for episode in range(train_episodes_per_iteration):
        
        players, boards, probs, winners = self_play(env, agent)
        
        print('训练 {} 回合 {}: 收集到 {} 条经验'.format(
            iteration + iteration_old, episode, len(players)))
        
        ls_players.append(players)
        ls_boards.append(boards)
        ls_probs.append(probs)
        winners = tf.concat([winners]*len(players), 0)
        ls_winners.append(winners)
        
    # 利用经验进行学习
    trajectorys["player"] = tf.concat(ls_players, 0)
    trajectorys["board"] = tf.concat(ls_boards, 0)
    trajectorys["prob"] = tf.concat(ls_probs, 0)
    trajectorys["winner"] = tf.concat(ls_winners, 0)
    
    dataset = tf.data.Dataset.from_tensor_slices(trajectorys)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer_size,reshuffle_each_iteration=True)
    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    agent.learn(dataset)
    print('训练 {}：学习完成'.format(iteration + iteration_old))
    
    current_model_path = os.path.join(H5_PATH, f"model_{iteration + iteration_old:04}iterations.h5")
    agent.net.save(current_model_path)
    tf.saved_model.save(agent.net, os.path.join(SavedModel_PATH, f"{iteration + iteration_old:04}"))
    
    # 演示训练结果
    self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
训练 79 回合 0: 收集到 200 条经验
训练 79 回合 1: 收集到 224 条经验
训练 79 回合 2: 收集到 120 条经验
训练 79 回合 3: 收集到 232 条经验
训练 79 回合 4: 收集到 136 条经验
训练 79 回合 5: 收集到 224 条经验
训练 79 回合 6: 收集到 200 条经验
训练 79 回合 7: 收集到 232 条经验
训练 79 回合 8: 收集到 320 条经验
训练 79 回合 9: 收集到 184 条经验
训练 79 回合 10: 收集到 128 条经验
训练 79 回合 11: 收集到 248 条经验
训练 79 回合 12: 收集到 224 条经验
训练 79 回合 13: 收集到 248 条经验
训练 79 回合 14: 收集到 232 条经验
训练 79 回合 15: 收集到 304 条经验
训练 79 回合 16: 收集到 256 条经验
训练 79 回合 17: 收集到 240 条经验
训练 79 回合 18: 收集到 160 条经验
训练 79 回合 19: 收集到 320 条经验
训练 79：学习完成
INFO:tensorflow:Assets written to: ./saved_model/Gomoku-v0_tf_21/0079/assets
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 1)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+o+++++
第 1 步： 玩家 -1， 动作 (2, 1)
+++++++
+++++++
+x+++++
++

o++ox+x
+++o+ox
++++o++
++x++++
+++++++
++++x++
+o+x+++
第 12 步： 玩家 1， 动作 (5, 6)
o++ox+x
+++o+ox
++++o++
++x++++
+++++++
++++x+o
+o+x+++
第 13 步： 玩家 -1， 动作 (3, 3)
o++ox+x
+++o+ox
++++o++
++xx+++
+++++++
++++x+o
+o+x+++
第 14 步： 玩家 1， 动作 (6, 4)
o++ox+x
+++o+ox
++++o++
++xx+++
+++++++
++++x+o
+o+xo++
第 15 步： 玩家 -1， 动作 (0, 2)
o+xox+x
+++o+ox
++++o++
++xx+++
+++++++
++++x+o
+o+xo++
第 16 步： 玩家 1， 动作 (2, 0)
o+xox+x
+++o+ox
o+++o++
++xx+++
+++++++
++++x+o
+o+xo++
第 17 步： 玩家 -1， 动作 (5, 0)
o+xox+x
+++o+ox
o+++o++
++xx+++
+++++++
x+++x+o
+o+xo++
第 18 步： 玩家 1， 动作 (5, 1)
o+xox+x
+++o+ox
o+++o++
++xx+++
+++++++
xo++x+o
+o+xo++
第 19 步： 玩家 -1， 动作 (3, 6)
o+xox+x
+++o+ox
o+++o++
++xx++x
+++++++
xo++x+o
+o+xo++
第 20 步： 玩家 1， 动作 (4, 0)
o+xox+x
+++o+ox
o+++o++
++xx++x
o++++++
xo++x+o
+o+xo++
第 21 步： 玩家 -1， 动作 (1, 4)
o+xox+x
+++oxox
o+++o++
++xx++x
o++++++
xo++x+o
+o+xo++
第 22 步： 玩家 1， 动作 (4, 2)
o+xox+x
+++oxox
o+++o++
++xx++x
o+o++++
xo++x+o
+o+xo++
第 23 步： 玩家 -1， 动作 (0, 1)
oxxox+x
+++oxox
o+++o++
++xx++x
o+

+++++++
++oox++
+x+x+++
+++o+++
+o+++x+
x++o+++
o++++++
第 11 步： 玩家 -1， 动作 (4, 2)
+++++++
++oox++
+x+x+++
+++o+++
+ox++x+
x++o+++
o++++++
第 12 步： 玩家 1， 动作 (5, 5)
+++++++
++oox++
+x+x+++
+++o+++
+ox++x+
x++o+o+
o++++++
第 13 步： 玩家 -1， 动作 (4, 4)
+++++++
++oox++
+x+x+++
+++o+++
+ox+xx+
x++o+o+
o++++++
第 14 步： 玩家 1， 动作 (4, 3)
+++++++
++oox++
+x+x+++
+++o+++
+oxoxx+
x++o+o+
o++++++
第 15 步： 玩家 -1， 动作 (0, 5)
+++++x+
++oox++
+x+x+++
+++o+++
+oxoxx+
x++o+o+
o++++++
第 16 步： 玩家 1， 动作 (1, 6)
+++++x+
++oox+o
+x+x+++
+++o+++
+oxoxx+
x++o+o+
o++++++
第 17 步： 玩家 -1， 动作 (6, 1)
+++++x+
++oox+o
+x+x+++
+++o+++
+oxoxx+
x++o+o+
ox+++++
第 18 步： 玩家 1， 动作 (0, 4)
++++ox+
++oox+o
+x+x+++
+++o+++
+oxoxx+
x++o+o+
ox+++++
第 19 步： 玩家 -1， 动作 (4, 0)
++++ox+
++oox+o
+x+x+++
+++o+++
xoxoxx+
x++o+o+
ox+++++
第 20 步： 玩家 1， 动作 (0, 1)
+o++ox+
++oox+o
+x+x+++
+++o+++
xoxoxx+
x++o+o+
ox+++++
第 21 步： 玩家 -1， 动作 (3, 0)
+o++ox+
++oox+o
+x+x+++
x++o+++
xoxoxx+
x++o+o+
ox+++++
第 22 步： 玩家 1， 动作 (6, 2)
+o++ox+
++oox+o
+x+x+++
x++o+++
xo

+++++++
+++oo+x
++++xx+
x++++++
o+o++++
+o+++++
+++++++
第 9 步： 玩家 -1， 动作 (0, 4)
++++x++
+++oo+x
++++xx+
x++++++
o+o++++
+o+++++
+++++++
第 10 步： 玩家 1， 动作 (6, 1)
++++x++
+++oo+x
++++xx+
x++++++
o+o++++
+o+++++
+o+++++
第 11 步： 玩家 -1， 动作 (3, 4)
++++x++
+++oo+x
++++xx+
x+++x++
o+o++++
+o+++++
+o+++++
第 12 步： 玩家 1， 动作 (5, 5)
++++x++
+++oo+x
++++xx+
x+++x++
o+o++++
+o+++o+
+o+++++
第 13 步： 玩家 -1， 动作 (4, 1)
++++x++
+++oo+x
++++xx+
x+++x++
oxo++++
+o+++o+
+o+++++
第 14 步： 玩家 1， 动作 (0, 3)
+++ox++
+++oo+x
++++xx+
x+++x++
oxo++++
+o+++o+
+o+++++
第 15 步： 玩家 -1， 动作 (1, 2)
+++ox++
++xoo+x
++++xx+
x+++x++
oxo++++
+o+++o+
+o+++++
第 16 步： 玩家 1， 动作 (0, 6)
+++ox+o
++xoo+x
++++xx+
x+++x++
oxo++++
+o+++o+
+o+++++
第 17 步： 玩家 -1， 动作 (2, 6)
+++ox+o
++xoo+x
++++xxx
x+++x++
oxo++++
+o+++o+
+o+++++
第 18 步： 玩家 1， 动作 (5, 6)
+++ox+o
++xoo+x
++++xxx
x+++x++
oxo++++
+o+++oo
+o+++++
第 19 步： 玩家 -1， 动作 (5, 2)
+++ox+o
++xoo+x
++++xxx
x+++x++
oxo++++
+ox++oo
+o+++++
第 20 步： 玩家 1， 动作 (4, 3)
+++ox+o
++xoo+x
++++xxx
x+++x++
oxo

In [6]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

#env = KInARowEnv(3,3)
env = KInARowEnv(7,5)

def categorical_crossentropy_2d(y_true:tf.Tensor,
                                y_pred:tf.Tensor) -> tf.keras.losses.Loss:
    labels = tf.reshape(y_true, [-1, env.board.size])
    preds = tf.reshape(y_pred, [-1,env.board.size])
    return tf.keras.losses.categorical_crossentropy(labels, preds)

#current_model_path = os.path.join(H5_PATH, f"model_{28:04}iterations.h5")
agent = AlphaZeroAgent(env=env, load=current_model_path)

self_play_eval(env, agent)

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:CPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:CPU:0'
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
第 0 步： 玩家 1， 动作 (6, 1)
+++++++
+++++++
+++++++
+++++++
+++++++
+++++++
+o+++++
第 1 步： 玩家 -1， 动作 (4, 0)
+++++++
+++++++
+++++++
+++++++
x++++++
+++++++
+o+++++
第 2 步： 玩家 1， 动作 (4, 1)
+++++++
+++++++
+++++++
+++++++
xo+++++
+++++++
+o+++++
第 3 步： 玩家 -1， 动作 (6, 3)
+++++++
+++++++
+++++++
+++++++
xo+++++
+++++++
+o+x+++
第 4 步： 玩家 1， 动作 (3, 5)
+++++++
+++++++
+++++++
+++++o+
xo+++++
+++++++
+o+x+++
第 5 步： 玩家 -1， 动作 (2, 2)
+++++++
+++++++
++x++++
+++++o+
xo+++++
+++++++
+o+x+++
第 6 步： 玩家 1， 动作 (5, 4)
+++++++
+++++++
++x++++
+++++o+
xo+++++
++++o++
+o+x+++
第 7 步： 玩家 -1， 动作 (3, 4)
+++++++
+++++++
++x++++
++++xo+
xo+++++
++++o++
+o+x+++
第 8 步： 玩家 1， 动作 (1, 3)
+++++++
+++o+++
++x++++
++++xo+
xo+++++
++++o++
